# Equity Research Report Generator

A comprehensive fundamental analysis tool for generating institutional-quality equity research reports.

## Features
- Company overview and financial data collection (via yfinance)
- Comprehensive financial ratio analysis
- Peer comparison and relative valuation
- Sentiment and momentum analysis
- Interactive visualizations
- Automated scoring and recommendation system

---

## Configuration

**Change the ticker below to analyze any stock:**

In [ ]:
# =============================================================================
# CONFIGURATION - CHANGE THIS TO ANALYZE A DIFFERENT STOCK
# =============================================================================

TICKER = "NVDA"  # Change this to any valid stock ticker

# Examples:
# TICKER = "AAPL"   # Apple
# TICKER = "MSFT"   # Microsoft
# TICKER = "GOOGL"  # Alphabet
# TICKER = "AMZN"   # Amazon
# TICKER = "META"   # Meta
# TICKER = "TSLA"   # Tesla
# TICKER = "JPM"    # JPMorgan
# TICKER = "JNJ"    # Johnson & Johnson

print(f"Analyzing: {TICKER}")

## Setup

In [ ]:
# Import required libraries
import sys
import os
import warnings
warnings.filterwarnings('ignore')

# Import custom modules
from src.data_collector import DataCollector
from src.financial_analysis import FinancialAnalyzer
from src.peer_comparison import PeerComparison
from src.sentiment_analysis import SentimentAnalyzer
from src.visualizations import ChartGenerator
from src.report_generator import ReportGenerator, ScoringEngine

# For interactive charts
import plotly.io as pio
pio.renderers.default = 'notebook'

print("Setup complete!")

---
## Phase 1: Data Collection

In [ ]:
# Collect all data for the ticker
print(f"Collecting data for {TICKER}...")
collector = DataCollector(TICKER)
data = collector.get_all_data()

# Display company overview
overview = data['overview']
print(f"\n{'='*60}")
print(f"Company: {overview['name']}")
print(f"Sector: {overview['sector']}")
print(f"Industry: {overview['industry']}")
print(f"Market Cap: ${overview['market_cap']:,.0f}" if overview['market_cap'] else "Market Cap: N/A")
print(f"Current Price: ${overview['current_price']:.2f}" if overview['current_price'] else "Price: N/A")
print(f"{'='*60}")

In [ ]:
# Display business description
print("Business Summary:")
print("-" * 60)
description = overview.get('description', 'N/A')
# Wrap text for better display
import textwrap
print(textwrap.fill(description[:1000] + "..." if len(description) > 1000 else description, width=80))

In [ ]:
# Display key statistics
stats = data['key_statistics']
print("\nKey Statistics:")
print("-" * 40)

key_stats = [
    ('P/E (Trailing)', stats.get('pe_trailing')),
    ('P/E (Forward)', stats.get('pe_forward')),
    ('PEG Ratio', stats.get('peg_ratio')),
    ('Price/Book', stats.get('price_to_book')),
    ('EV/EBITDA', stats.get('ev_to_ebitda')),
    ('Profit Margin', stats.get('profit_margin')),
    ('ROE', stats.get('roe')),
    ('Beta', stats.get('beta')),
]

for name, value in key_stats:
    if value is not None:
        if 'Margin' in name or 'ROE' in name:
            print(f"{name}: {value:.1%}")
        else:
            print(f"{name}: {value:.2f}")

---
## Phase 2: Financial Analysis

In [ ]:
# Perform comprehensive financial analysis
analyzer = FinancialAnalyzer(data)
analysis = analyzer.get_comprehensive_analysis()
analysis['historical_prices'] = data.get('historical_prices')

print("Financial Analysis Complete!")

In [ ]:
# Display valuation metrics
from IPython.display import display, HTML
import pandas as pd

val = analysis['valuation']
valuation_data = {
    'Metric': ['P/E (TTM)', 'P/E (Forward)', 'PEG Ratio', 'P/B', 'P/S', 'EV/EBITDA', 'EV/Revenue', 'P/FCF'],
    'Value': [
        f"{val.get('pe_trailing'):.2f}" if val.get('pe_trailing') else 'N/A',
        f"{val.get('pe_forward'):.2f}" if val.get('pe_forward') else 'N/A',
        f"{val.get('peg_ratio'):.2f}" if val.get('peg_ratio') else 'N/A',
        f"{val.get('price_to_book'):.2f}" if val.get('price_to_book') else 'N/A',
        f"{val.get('price_to_sales'):.2f}" if val.get('price_to_sales') else 'N/A',
        f"{val.get('ev_to_ebitda'):.2f}" if val.get('ev_to_ebitda') else 'N/A',
        f"{val.get('ev_to_revenue'):.2f}" if val.get('ev_to_revenue') else 'N/A',
        f"{val.get('price_to_fcf'):.2f}" if val.get('price_to_fcf') else 'N/A',
    ]
}

print("Valuation Metrics:")
display(pd.DataFrame(valuation_data))

In [ ]:
# Display profitability metrics
prof = analysis['profitability']

profitability_data = {
    'Metric': ['Gross Margin', 'Operating Margin', 'Net Margin', 'ROE', 'ROA', 'ROIC'],
    'Value': [
        f"{prof.get('gross_margin'):.1%}" if prof.get('gross_margin') else 'N/A',
        f"{prof.get('operating_margin'):.1%}" if prof.get('operating_margin') else 'N/A',
        f"{prof.get('net_margin'):.1%}" if prof.get('net_margin') else 'N/A',
        f"{prof.get('roe'):.1%}" if prof.get('roe') else 'N/A',
        f"{prof.get('roa'):.1%}" if prof.get('roa') else 'N/A',
        f"{prof.get('roic'):.1%}" if prof.get('roic') else 'N/A',
    ]
}

print("Profitability Metrics:")
display(pd.DataFrame(profitability_data))

In [ ]:
# Display growth metrics
growth = analysis['growth']

growth_data = {
    'Metric': ['Revenue Growth (YoY)', 'Revenue CAGR (3Y)', 'EPS Growth (YoY)', 'FCF Growth (YoY)'],
    'Value': [
        f"{growth.get('revenue_growth_1y'):.1%}" if growth.get('revenue_growth_1y') else 'N/A',
        f"{growth.get('revenue_cagr_3y'):.1%}" if growth.get('revenue_cagr_3y') else 'N/A',
        f"{growth.get('eps_growth_1y'):.1%}" if growth.get('eps_growth_1y') else 'N/A',
        f"{growth.get('fcf_growth_1y'):.1%}" if growth.get('fcf_growth_1y') else 'N/A',
    ]
}

print("Growth Metrics:")
display(pd.DataFrame(growth_data))

---
## Phase 3: Peer Comparison

In [ ]:
# Perform peer comparison
peer_comp = PeerComparison(TICKER, data)
peers = peer_comp.identify_peers()
print(f"Identified peers: {', '.join(peers)}")

print("\nCollecting peer data...")
peer_comp.collect_peer_data()
peer_summary = peer_comp.get_peer_summary()
print("Peer comparison complete!")

In [ ]:
# Display peer comparison table
print("\nPeer Comparison Matrix:")
display(peer_summary['formatted_comparison'])

In [ ]:
# Display relative valuation assessment
rel_val = peer_summary['relative_valuation']
justification = peer_summary['valuation_justification']

print("Relative Valuation Assessment:")
print("=" * 50)
print(f"Status: {rel_val.get('assessment', 'N/A')}")
print(f"Detail: {rel_val.get('assessment_detail', 'N/A')}")
print(f"\nConclusion: {justification.get('conclusion', 'N/A')}")

if justification.get('factors_supporting'):
    print("\nFactors Supporting:")
    for f in justification['factors_supporting']:
        print(f"  + {f}")

if justification.get('factors_against'):
    print("\nFactors Against:")
    for f in justification['factors_against']:
        print(f"  - {f}")

---
## Phase 4: Sentiment Analysis

In [ ]:
# Perform sentiment analysis
sentiment_analyzer = SentimentAnalyzer(TICKER, data)
sentiment = sentiment_analyzer.get_sentiment_summary()

print("Sentiment Analysis Complete!")

In [ ]:
# Display analyst recommendations
analyst = sentiment['analyst_recommendations']

print("Analyst Recommendations:")
print("=" * 50)
print(f"Consensus: {analyst.get('consensus', 'N/A')}")
print(f"Total Analysts: {analyst.get('total_analysts', 'N/A')}")
print(f"Mean Rating: {analyst.get('mean_rating', 'N/A'):.2f}" if analyst.get('mean_rating') else "Mean Rating: N/A")

if analyst.get('recommendation_breakdown'):
    print("\nBreakdown:")
    for rating, count in analyst['recommendation_breakdown'].items():
        print(f"  {rating}: {count}")

pt = analyst.get('price_targets', {})
if pt.get('target_mean'):
    print(f"\nPrice Targets:")
    print(f"  Current: ${pt.get('current_price', 0):.2f}")
    print(f"  Mean Target: ${pt.get('target_mean', 0):.2f}")
    print(f"  Range: ${pt.get('target_low', 0):.2f} - ${pt.get('target_high', 0):.2f}")
    if pt.get('upside_pct'):
        print(f"  Implied Upside: {pt['upside_pct']:.1f}%")

In [ ]:
# Display momentum analysis
momentum = sentiment['momentum']

print("Technical Momentum:")
print("=" * 50)
print(f"Overall: {momentum.get('overall_momentum', 'N/A')}")
print(f"Score: {momentum.get('momentum_score', 'N/A')}/10")

print("\nSignals:")
for signal in momentum.get('technical_signals', []):
    print(f"  - {signal}")

In [ ]:
# Overall sentiment
print("\n" + "=" * 50)
print(f"OVERALL SENTIMENT: {sentiment.get('overall_sentiment', 'N/A')}")
print(f"SENTIMENT SCORE: {sentiment.get('overall_sentiment_score', 'N/A')}/10")
print("=" * 50)

---
## Phase 5: Visualizations

In [ ]:
# Initialize chart generator
chart_gen = ChartGenerator(TICKER, 'output/charts')

In [ ]:
# Price Chart with Moving Averages
price_chart = chart_gen.create_price_chart(data['historical_prices'], years=2)
if price_chart:
    price_chart.show()

In [ ]:
# Financial Trends Chart
trends_chart = chart_gen.create_financial_trends_chart(analysis['growth'])
if trends_chart:
    trends_chart.show()

In [ ]:
# Margin Analysis Chart
margin_chart = chart_gen.create_margin_analysis_chart(analysis['profitability'])
if margin_chart:
    margin_chart.show()

In [ ]:
# Peer Valuation Comparison
peer_chart = chart_gen.create_peer_valuation_chart(peer_summary['comparison_matrix'])
if peer_chart:
    peer_chart.show()

In [ ]:
# Growth vs Valuation Scatter
scatter_chart = chart_gen.create_growth_vs_valuation_scatter(peer_summary['comparison_matrix'])
if scatter_chart:
    scatter_chart.show()

In [ ]:
# Sentiment Gauge
sentiment_gauge = chart_gen.create_sentiment_gauge(
    sentiment.get('overall_sentiment_score', 5),
    sentiment.get('overall_sentiment', 'Neutral')
)
if sentiment_gauge:
    sentiment_gauge.show()

---
## Phase 6: Investment Recommendation

In [ ]:
# Generate report and scores
report_gen = ReportGenerator(TICKER, data, analysis, peer_summary, sentiment)
scores = report_gen.get_scores()

print("Scoring Analysis Complete!")

In [ ]:
# Display scoring matrix
print("Investment Scoring Matrix:")
print("=" * 60)

scoring_data = []
for cat, cat_scores in scores['categories'].items():
    scoring_data.append({
        'Category': cat,
        'Weight': f"{cat_scores['weight']:.0%}",
        'Score': f"{cat_scores['score']:.1f}",
        'Weighted': f"{cat_scores['weighted']:.2f}"
    })

scoring_df = pd.DataFrame(scoring_data)
display(scoring_df)

print(f"\nTOTAL SCORE: {scores['total_score']}/10")
print(f"RECOMMENDATION: {scores['recommendation']}")

In [ ]:
# Display score breakdown chart
score_chart = chart_gen.create_recommendation_score_chart(scores)
if score_chart:
    score_chart.show()

In [ ]:
# Display scoring factors
print("\nScoring Factors by Category:")
print("=" * 60)

for category, factors in scores['all_factors'].items():
    if factors:
        print(f"\n{category.upper()}:")
        for factor in factors:
            print(f"  - {factor}")

---
## Bull and Bear Cases

In [ ]:
from src.report_generator import ThesisGenerator

thesis_gen = ThesisGenerator(analysis, peer_summary, sentiment, scores)

bull_case = thesis_gen.generate_bull_case()
bear_case = thesis_gen.generate_bear_case()
key_metrics = thesis_gen.generate_key_metrics_to_monitor()

print("BULL CASE:")
print("=" * 50)
for i, point in enumerate(bull_case, 1):
    print(f"{i}. {point}")

print("\nBEAR CASE:")
print("=" * 50)
for i, point in enumerate(bear_case, 1):
    print(f"{i}. {point}")

print("\nKEY METRICS TO MONITOR:")
print("=" * 50)
for metric in key_metrics:
    print(f"- {metric}")

---
## Generate Full Report

In [ ]:
# Generate and save the full report
import os
os.makedirs('output', exist_ok=True)

report = report_gen.generate_full_report()

report_path = f"output/{TICKER}_research_report.md"
with open(report_path, 'w') as f:
    f.write(report)

print(f"Full report saved to: {report_path}")
print(f"\nReport length: {len(report):,} characters")

In [ ]:
# Display the report
from IPython.display import Markdown
display(Markdown(report))

---
## Save Charts

In [ ]:
# Save all charts
import os
os.makedirs('output/charts', exist_ok=True)

charts = chart_gen.save_all_charts(
    analysis=analysis,
    peer_comparison=peer_summary['comparison_matrix'],
    sentiment=sentiment,
    scores=scores,
    format='png'
)

print(f"Saved {len(charts)} charts:")
for name, path in charts.items():
    print(f"  - {name}: {path}")

---
## Summary

In [ ]:
print("\n" + "=" * 60)
print(f"  ANALYSIS COMPLETE: {TICKER}")
print("=" * 60)
print(f"\n  Company: {analysis.get('company_name', 'N/A')}")
print(f"  Sector: {analysis.get('sector', 'N/A')}")
print(f"  Industry: {analysis.get('industry', 'N/A')}")
print(f"\n  Current Price: ${overview.get('current_price', 0):.2f}")
print(f"  Market Cap: ${overview.get('market_cap', 0)/1e9:.1f}B")
print(f"\n  Investment Score: {scores['total_score']}/10")
print(f"  Recommendation: {scores['recommendation']}")
print(f"\n  Report: {report_path}")
print("\n" + "=" * 60)

---

## Disclaimer

*This report is for informational purposes only and does not constitute financial advice. Always conduct your own research and consult with a qualified financial advisor before making investment decisions.*